In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xgi
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from draw import *
from sod import *

In [ ]:
dataset = "email-enron"
max_order = 2

H = xgi.load_xgi_data(dataset, max_order=max_order)
H.cleanup()

In [3]:
pos = xgi.pairwise_spring_layout(H, seed=0)
pos = xgi.pca_transform(pos)
sf = H.nodes.local_simplicial_fraction.asnumpy()
es = H.nodes.local_edit_simpliciality.asnumpy()
fes = H.nodes.local_face_edit_simpliciality.asnumpy()

In [4]:
cmap = cm.RdYlGn
cmap.set_bad("lightgrey")
nan_color = "lightgrey"

c_sf = np.empty(H.num_nodes, dtype=object)
for n, s in enumerate(sf):
    c_sf[n] = cmap(s)

c_es = np.empty(H.num_nodes, dtype=object)
for n, s in enumerate(es):
    c_es[n] = cmap(s)

c_fes = np.empty(H.num_nodes, dtype=object)
for n, s in enumerate(fes):
    c_fes[n] = cmap(s)

In [ ]:
set_fonts()
node_size = 5
inset_node_size = 5
nbins = 20

plt.figure(figsize=(8, 8), dpi=50)
plt.subplots_adjust(hspace=0.5)

plt.subplot(221)
plt.title("Simplicial Fraction\n")

n, bins, patches = plt.hist(sf, bins=np.linspace(0, 1, nbins))
plt.xlim([0, 1])
plt.xticks([0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1])
plt.ylim([0, 40])
plt.yticks([0, 20, 40])
plt.xlabel(r"$\sigma_{\mathrm{SF}}$")
plt.ylabel("Number of nodes")

bin_centers = 0.5 * (bins[:-1] + bins[1:])

for c, p in zip(bin_centers, patches):
    plt.setp(p, "facecolor", cmap(c))
sns.despine()

plt.subplot(222)
plt.axis("off")
inset_ax = plt.gca().inset_axes([-0.3, -0.25, 1.5, 1.75])
inset_ax.set_axis_off()
xgi.draw_hyperedges(
    H,
    ax=inset_ax,
    pos=pos,
    hyperedge_alpha=0.1,
    zorder=-1,
    dyad_lw=0.5,
    dyad_color="#333",
)
xgi.draw_nodes(
    H,
    ax=inset_ax,
    pos=pos,
    node_size=node_size,
    node_fc=c_sf,
    node_lw=0.5,
    alpha=0.5,
    zorder=10,
)


plt.subplot(223)
plt.title("Edit Simpliciality\n")
n, bins, patches = plt.hist(es, bins=np.linspace(0, 1, nbins))
plt.xlim([0, 1])
plt.xticks([0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1])
plt.ylim([0, 40])
plt.yticks([0, 20, 40])
plt.xlabel(r"$\sigma_{\mathrm{ES}}$")
plt.ylabel("Number of nodes")

for c, p in zip(bin_centers, patches):
    plt.setp(p, "facecolor", cmap(c))
sns.despine()

inset_ax = plt.gca().inset_axes([0.05, 0.5, 0.5, 0.5])
xgi.draw_hyperedges(
    H, pos, ax=inset_ax, hyperedge_alpha=0.1, zorder=-1, dyad_lw=0.5, dyad_color="#333"
)
xgi.draw_nodes(
    H,
    pos,
    ax=inset_ax,
    node_size=inset_node_size,
    node_fc=c_es,
    node_lw=0.5,
    alpha=0.5,
    zorder=10,
)


plt.subplot(224)
plt.title("Face Edit Simpliciality\n")
n, bins, patches = plt.hist(fes, bins=np.linspace(0, 1, nbins))
plt.xlim([0, 1])
plt.xticks([0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1])
plt.ylim([0, 40])
plt.yticks([0, 20, 40])
plt.xlabel(r"$\sigma_{\mathrm{FES}}$")

for c, p in zip(bin_centers, patches):
    plt.setp(p, "facecolor", cmap(c))
sns.despine()

inset_ax = plt.gca().inset_axes([0.05, 0.5, 0.5, 0.5])
xgi.draw_hyperedges(
    H, pos, ax=inset_ax, hyperedge_alpha=0.1, zorder=-1, dyad_lw=0.5, dyad_color="#333"
)
xgi.draw_nodes(
    H,
    pos,
    ax=inset_ax,
    node_size=inset_node_size,
    node_fc=c_fes,
    node_lw=0.5,
    alpha=0.5,
    zorder=10,
)


plt.figtext(0.1, 0.9375, "A")
plt.figtext(0.1, 0.4565, "B")
plt.figtext(0.5625, 0.4565, "C")

plt.savefig(f"Figures/Fig3/{dataset}_local_simpliciality.png", dpi=1000)
plt.savefig(f"Figures/Fig3/{dataset}_local_simpliciality.pdf", dpi=1000)
plt.show()